In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Read in og image and change to HLS

In [ ]:
image1 = cv2.imread('../data/images/StockPhoto1.jpg')
hls_image = cv2.cvtColor(image1, cv2.COLOR_BGR2HLS)

Remove green screen

In [ ]:
# Define green hue range in HLS
lower_green = np.array([35, 40, 40])        # H, L, S
upper_green = np.array([85, 255, 255])

# Create a mask where green pixels are white
green_mask = cv2.inRange(hls_image, lower_green, upper_green)

# Invert mask to keep everything that's NOT green
mask_no_green = cv2.bitwise_not(green_mask)

# Use the inverted mask to extract non-green parts of the image
result = cv2.bitwise_and(image1, image1, mask=mask_no_green)

# OPTIONAL: Make green pixels fully transparent (RGBA output)
result_rgba = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result_rgba[green_mask == 255] = [0, 0, 0, 0]   # Set green areas to transparent

# Save image as rgba
cv2.imwrite('../outputs/removed_greenscreen.png', result_rgba)

Read in new image and change to HLS

In [ ]:
image2 = cv2.imread('../outputs/removed_greenscreen.png')
hls = cv2.cvtColor(image2, cv2.COLOR_BGR2HLS)

Detect subtle greens

In [ ]:
hue, lightness, saturation = cv2.split(hls)

# Green spill detection: greenish hue, low saturation, green dominance
green_hue = 60
hue_distance = np.abs(hue.astype(np.int16) - green_hue)
hue_distance = np.minimum(hue_distance, 180 - hue_distance)
greenness = 180 - hue_distance.astype(np.uint8)

# Mask for subtle green spill
_, spill_mask = cv2.threshold(greenness, 150, 255, cv2.THRESH_BINARY)


Clean up the edges (Despill)

In [ ]:
# Convert image to float32 for precise edits
image_float = image2.astype(np.float32)

# Apply spill factor (modify green channel)
spill_factor = 0.0  # Lower the number, the harsher the green removal is
image_float[:, :, 1][spill_mask == 255] *= spill_factor # green channel

# Clip and convert back
despilled = np.clip(image_float, 0, 255).astype(np.uint8)
cv2.imwrite('../outputs/despilled.png', despilled)